In [1]:
import os
import time

import pinocchio as pin
import numpy as np
import scipy

from ompl import base as ob
from ompl import geometric as og

In [2]:
model, collision_model, visual_model = pin.buildModelsFromUrdf(
    "../model/walli_arm.urdf",
    package_dirs=[
        "/home/keseterg/Documents/Robojackets/urc_ws/src/urc_arm/model/meshes"
    ],
)
collision_model = pin.buildGeomFromUrdf(
    model, "../model/walli_arm.urdf",
    pin.GeometryType.COLLISION,
    package_dirs=[
        "/home/keseterg/Documents/Robojackets/urc_ws/src/urc_arm/model/meshes",
    ]
)
collision_model.addAllCollisionPairs()
print(f"{len(collision_model.collisionPairs)}")
robot = pin.RobotWrapper(model, collision_model, visual_model, True)

robot.setVisualizer(pin.visualize.MeshcatVisualizer())
robot.initViewer()
robot.loadViewerModel("pinocchio")
robot.display(robot.q0)
robot.displayVisuals(True)
robot.displayCollisions(True)

28
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [9]:
robot.updateGeometryPlacements(robot.q0, True)
q_test = robot.q0
q_test[4] = 2.0
robot.forwardKinematics(q_test)
robot.display(q_test)
pin.computeCollisions(
    robot.model, robot.data,
    robot.collision_model, robot.collision_data,
    robot.q0, False
)

True

In [10]:
for k in range(len(robot.collision_model.collisionPairs)): 
  cr = robot.collision_data.collisionResults[k]
  cp = robot.collision_model.collisionPairs[k]
  print("collision pair:",cp.first,",",cp.second,"- collision:","Yes" if cr.isCollision() else "No")

collision pair: 0 , 1 - collision: No
collision pair: 0 , 2 - collision: No
collision pair: 0 , 3 - collision: No
collision pair: 0 , 4 - collision: No
collision pair: 0 , 5 - collision: No
collision pair: 0 , 6 - collision: No
collision pair: 0 , 7 - collision: No
collision pair: 1 , 2 - collision: No
collision pair: 1 , 3 - collision: No
collision pair: 1 , 4 - collision: No
collision pair: 1 , 5 - collision: No
collision pair: 1 , 6 - collision: No
collision pair: 1 , 7 - collision: No
collision pair: 2 , 3 - collision: Yes
collision pair: 2 , 4 - collision: No
collision pair: 2 , 5 - collision: No
collision pair: 2 , 6 - collision: No
collision pair: 2 , 7 - collision: No
collision pair: 3 , 4 - collision: No
collision pair: 3 , 5 - collision: No
collision pair: 3 , 6 - collision: No
collision pair: 3 , 7 - collision: No
collision pair: 4 , 5 - collision: No
collision pair: 4 , 6 - collision: No
collision pair: 4 , 7 - collision: No
collision pair: 5 , 6 - collision: No
collision p

In [12]:
space = ob.SE3StateSpace()
bounds = ob.RealVectorBounds(3)
bounds.setLow(-1)
bounds.setHigh(1)
space.setBounds(bounds)

In [17]:
def is_valid(state:ob.State) -> bool:
    pos = state[0]
    vel = state[1]
    return True
space_info = ob.SpaceInformation(space)
space_info.setStateValidityChecker(ob.StateValidityCheckerFn(is_valid))

In [18]:
start_state = ob.State(space_info)
start_state.random()
goal_state = ob.State(space_info)
goal_state.random()

problem = ob.ProblemDefinition(space_info)
problem.setStartAndGoalStates(start_state, goal_state)

In [19]:
planner = og.RRTConnect(space_info)
planner.setProblemDefinition(problem)
planner.setup()
solved = planner.solve(1.0)

Info:    RRTConnect: Space information setup was not yet called. Calling now.
Debug:   RRTConnect: Planner range detected to be 1.006980
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 6 states (2 start + 4 goal)


In [21]:
print(solved)
print(problem.getSolutionPath())

Exact solution
Geometric path with 5 states
Compound state [
RealVectorState [-0.360901 0.26515 -0.248328]
SO3State [-0.793012 -0.14166 -0.525166 0.274344]
]
Compound state [
RealVectorState [-0.346296 0.58111 -0.243255]
SO3State [-0.331151 0.0543523 -0.938202 0.0846326]
]
Compound state [
RealVectorState [-0.249987 0.454789 -0.130016]
SO3State [-0.295474 0.0120669 -0.955262 0.00492807]
]
Compound state [
RealVectorState [0.0802967 0.0215834 0.258328]
SO3State [-0.1537 -0.131313 -0.943091 -0.264031]
]
Compound state [
RealVectorState [0.41058 -0.411622 0.646673]
SO3State [0.00546536 -0.259835 -0.824215 -0.503116]
]
